# Scrape CDEC historical to present reservoir storage data

CDEC gives reservior storage data at a temporal resolution as fine as the hour, but we're just going to grab their monthly reservoir storage product, which is all we need. Let's grab all the data from the 1974 water year ($t_0$ for C2VSimFG beta) through today!

In [1]:
# packages
library(tidyverse) # general purpose DS toolkit
library(rvest)     # for webscraping
library(viridis)   # colorblind-safe color palettes
library(readr)     # fast read/write functions

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.0     v purrr   0.2.5
v tibble  1.4.2     v dplyr   0.7.6
v tidyr   0.8.1     v stringr 1.3.1
v readr   1.1.1     v forcats 0.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Loading required package: xml2

Attaching package: 'rvest'

The following object is masked from 'package:purrr':

    pluck

The following object is masked from 'package:readr':

    guess_encoding

Loading required package: viridisLite


In [2]:
# scrape CDEC station codes
station_codes <- read_html('http://cdec.water.ca.gov/misc/resinfo.html') %>% 
  html_table() %>% 
  .[[1]] %>% 
  pull(ID)

# starting and ending time indices
start <- '1973-10-31'               # start of 1974 water year (C2VSimFG beta)
end   <- '2019-01-23'               # today

# constrct the url to query CDEC
url <- paste0('https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations=', 
       paste(station_codes, collapse = '%2C'),
       '&SensorNums=15&dur_code=M&Start=',
       start,
       '&End=',
       end)

# read the generated webpage, clean the date column, and omit missing values for the present day
df <- read.csv(url)

df <- df %>% 
  mutate(year = substr(df$OBS.DATE, 1, 4), 
         month = substr(df$OBS.DATE, 5, 6), 
         day = substr(df$OBS.DATE, 7, 8), 
         ymd = lubridate::ymd(paste(year, month, day, sep = "-"))) %>% 
  filter(VALUE != "---") %>% 
  mutate(VALUE = as.numeric(VALUE))

In [3]:
# view what we've got
df

STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE.TIME,OBS.DATE,VALUE,DATA_FLAG,UNITS,year,month,day,ymd
ALM,M,15,STORAGE,19731101 0000,19731101 0000,53609,,AF,1973,11,01,1973-11-01
ALM,M,15,STORAGE,19731201 0000,19731201 0000,54230,,AF,1973,12,01,1973-12-01
ALM,M,15,STORAGE,19740101 0000,19740101 0000,738,,AF,1974,01,01,1974-01-01
ALM,M,15,STORAGE,19740201 0000,19740301 0000,260,,AF,1974,03,01,1974-03-01
ALM,M,15,STORAGE,19740301 0000,19740228 0000,1710,,AF,1974,02,28,1974-02-28
ALM,M,15,STORAGE,19740401 0000,19740401 0000,1411,,AF,1974,04,01,1974-04-01
ALM,M,15,STORAGE,19740501 0000,19740501 0000,2345,,AF,1974,05,01,1974-05-01
ALM,M,15,STORAGE,19740601 0000,19740601 0000,2419,,AF,1974,06,01,1974-06-01
ALM,M,15,STORAGE,19740701 0000,19740701 0000,1609,,AF,1974,07,01,1974-07-01
ALM,M,15,STORAGE,19740801 0000,19740731 0000,199,,AF,1974,07,31,1974-07-31


***  

## Visualize  

There are 199 station codes, so let's just look at the monthly storage in 3 reservoirs that feed the Central Valley: Folsom, Shasta, and Don Pedro.

In [4]:
df %>%
  filter(STATION_ID %in% c("FOL", "SHA", "DNP")) %>% 
  ggplot(aes(ymd, VALUE, color = STATION_ID)) + 
  geom_line() + 
  scale_color_viridis_d() +
  labs(title = "Monthly Reservoir Storage",
       subtitle = "Don Pedro, Folsom, and Shasta Reservoirs",
       x = "Date", y = "Storage (AF)")

ERROR: Error in file(con, "rb"): cannot open the connection


plot without title

***  

## Export for ML  

These data still need to be filtered down to the subset of reservoirs that feed the Central Valley, so we make a note to do so, and export the data now just so we have it for later.

In [ ]:
#write_tsv(df, "C:/Users/rpauloo/Documents/Github/pred_gws/ml/predictors/cdec_reservoir_storage.tsv")

In [8]:
library(plotly)
plot_ly(filter(df, STATION_ID == "ALM"), x = ~ymd, y = ~VALUE)

No trace type specified:
  Based on info supplied, a 'scatter' trace seems appropriate.
  Read more about this trace type -> https://plot.ly/r/reference/#scatter
No scatter mode specifed:
  Setting the mode to markers
  Read more about this attribute -> https://plot.ly/r/reference/#scatter-mode
Warning message in file(open = "w+b", encoding = "UTF-8"):
"cannot open file 'C:\Users\rpauloo\AppData\Local\Temp\RtmpWQZFKe\Rf503014c6273f': No such file or directory"ERROR while rich displaying an object: Error in file(open = "w+b", encoding = "UTF-8"): cannot open the connection

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallin

HTML widgets cannot be represented in plain text (need html)